# Aggregation for income table
* Income table shows number of hh per tract and the median hh income
* When aggregating from tract up to larger geographies, we need to make sure that we're deriving the corrected weighted median hh income.
    * (median hh income for tract) * (# hh in tract)
    * merge in crosswalk to other geography
    * allocate then sum over all tracts within CD, NC, etc
    * sum # hh within CD, NC, etc
    * calculate the weighted median hh income
* Need to figure out how to reconstruct median hh income at aggregated level
* [Dept of Finance example of using income range](http://www.dof.ca.gov/Forecasting/Demographics/Census_Data_Center_Network/documents/How_to_Recalculate_a_Median.pdf)

In [1]:
import numpy as np
import pandas as pd
import intake
import os

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')

## New: Use income ranges, aggregate, then recalculate the median

In [3]:
# Import Census tabular data
census = pd.read_parquet('s3://hcid-cdbg-project-ita-data/data/raw/raw_census_cleaned.parquet')

# Test this on 2016, 2017 for all census tracts
incomerange = census[(census.year >= 2016) & (census.table=='incomerange')]
incomerange_hh = census[(census.year >= 2016) & (census.table=='incomerange_hh')]

In [4]:
# Import crosswalks
council_districts = catalog.crosswalk_tracts_council_districts.read()
neighborhood_councils = catalog.crosswalk_tracts_neighborhood_councils.read()
zipcodes = catalog.crosswalk_tracts_zipcodes.read()
congressional_districts = catalog.crosswalk_tracts_congressional_districts.read()
neighborhoods = catalog.crosswalk_tracts_neighborhoods.read()

In [5]:
boundaries = {'council_districts': council_districts, 'neighborhood_councils': neighborhood_councils,
             'zipcodes': zipcodes, 'congressional_districts': congressional_districts, 'neighborhoods': neighborhoods}


# Loop through incomerange and incomerange_hh tables
income_dfs = {'incomerange': incomerange, 'incomerange_hh': incomerange_hh}

processed_dfs = {}

for key, value in boundaries.items():
    # Loop through incomerange and incomerange_hh tables, since they have the same structure.
    for filename, file in income_dfs.items():
        # Merge the incomerange table with each boundary 
        merged = pd.merge(file, value, on = 'GEOID', how = 'left', validate = 'm:1')
        merged.max_val = merged.max_val.fillna(0)
        # Allocate the num column according to however many CDs, NCs, etc each tract intersects with. 
        # Find the sum for num1, num2, ... columns. Then, append and take the sum again.
        n = merged.max_val.max().astype(int)
        uniform_id_col = 'ID'
        uniform_num_col = 'num' 
        aggregated = pd.DataFrame()
        # Depending on the boundary, tract might intersect with 1, 2,...,5 of the larger geographies.
        for i in range(1, n + 1):
            num_col = f"num{i}"
            allocate_col = f"allocate{i}"
            id_col = f"ID{i}"
            # Allocate the num column for all the various intersections.
            merged[num_col] = merged.num * merged[allocate_col]
            # Take the sum of the num column by CD, NC, etc.
            agg = merged.groupby([id_col, 'year', 'table', 'main_var', 'second_var']).agg({num_col: 'sum'}).reset_index()
            agg.rename(columns = {id_col: uniform_id_col, num_col: uniform_num_col}, inplace = True)
            # Append these sums together
            aggregated = aggregated.append(agg)
        # Take the sum again. For each CD, NC, etc, calculate the total # of hh and the total hh-weighted income
        aggregated2 = aggregated.groupby([uniform_id_col, 'year', 'table', 'main_var', 'second_var']).agg({uniform_num_col: 'sum'}).reset_index()
        # Round the number of households in each range, since allocating them results in decimal places
        final_df = f"{filename}_{key}"
        processed_dfs[final_df] = aggregated2

In [6]:
for key, value in processed_dfs.items():
    display(key)
    display(value.head())

'incomerange_council_districts'

,ID,year,table,main_var,second_var,num
0,1.0,2016,incomerange,amerind,gt200,0.000000
1,1.0,2016,incomerange,amerind,lt10,128.616755
2,1.0,2016,incomerange,amerind,r100to124,76.306859
3,1.0,2016,incomerange,amerind,r10to14,95.000000
4,1.0,2016,incomerange,amerind,r125to149,12.065514


'incomerange_hh_council_districts'

,ID,year,table,main_var,second_var,num
0,1.0,2016,incomerange_hh,families,gt200,1.880838e+02
1,1.0,2016,incomerange_hh,families,lt10,6.208629e+02
2,1.0,2016,incomerange_hh,families,meaninc,3.679704e+06
3,1.0,2016,incomerange_hh,families,medinc,2.716444e+06
4,1.0,2016,incomerange_hh,families,r100to149,4.972365e+02


'incomerange_neighborhood_councils'

...

,ID,year,table,main_var,second_var,num
0,625.0,2016,incomerange_hh,families,gt200,1.500082e+02
1,625.0,2016,incomerange_hh,families,lt10,1.246162e+01
2,625.0,2016,incomerange_hh,families,meaninc,1.123976e+06
3,625.0,2016,incomerange_hh,families,medinc,8.280120e+05
4,625.0,2016,incomerange_hh,families,r100to149,1.473142e+02


'incomerange_neighborhoods'

,ID,year,table,main_var,second_var,num
0,1.0,2016,incomerange,amerind,gt200,0.0
1,1.0,2016,incomerange,amerind,lt10,0.0
2,1.0,2016,incomerange,amerind,r100to124,0.0
3,1.0,2016,incomerange,amerind,r10to14,0.0
4,1.0,2016,incomerange,amerind,r125to149,0.0


'incomerange_hh_neighborhoods'

,ID,year,table,main_var,second_var,num
0,1.0,2016,incomerange_hh,families,gt200,7.194837
1,1.0,2016,incomerange_hh,families,lt10,58.332840
2,1.0,2016,incomerange_hh,families,meaninc,256299.174454
3,1.0,2016,incomerange_hh,families,medinc,175335.060505
4,1.0,2016,incomerange_hh,families,r100to149,37.850880


### Re-calculate median

In [7]:
test = processed_dfs['incomerange_council_districts']

In [9]:
# Define the order, so we can generate cumulative percents
incomerange_order = {'total': 1, 'lt10': 2, 'r10to14': 3, 'r15to19': 4, 'r20to24': 5,
           'r25to29': 6, 'r30to34': 7, 'r35to39': 8, 'r40to44': 9,  'r45to49': 10,
            'r50to59': 11, 'r60to74': 12, 'r75to99': 13, 'r100to124': 14, 'r125to149': 15,
            'r150to199': 16,  'gt200': 17}

incomerange_hh_order = {'total': 1, 'lt10': 2, 'r10to14': 3, 'r15to24': 4, 'r25to34': 5,
           'r35to49': 6,  'r50to74': 7, 'r75to99': 8, 'r100to149': 9, 'r150to199': 10,
           'gt200': 11}

In [10]:
test['order'] = test.second_var.map(incomerange_order)
test = test.sort_values(['ID', 'year', 'main_var', 'order'])

In [17]:
# Find the midpoint (total number of ppl / 2)
test['midpoint'] = test.apply(lambda row: row.num / 2 if row.second_var=='total' else np.nan, axis = 1)
test.midpoint = test.midpoint.fillna(test.groupby(['ID', 'year', 'table', 'main_var'])['midpoint'].transform('max')).round(2)

In [ ]:
# Tag the income range where this midpoint occurs
test['midpoint_range'] = test.apply(lambda row: row.order if row.midpoint <= row.num else np.nan, axis = 1)

In [20]:
test.order.max()

17

In [21]:
test[(test.main_var=='total') & (test.ID==1) & (test.year==2016)]

,ID,year,table,main_var,second_var,num,order,midpoint
152,1.0,2016,incomerange,total,total,82851.507414,1,41425.75
137,1.0,2016,incomerange,total,lt10,8855.899607,2,41425.75
139,1.0,2016,incomerange,total,r10to14,8361.659428,3,41425.75
142,1.0,2016,incomerange,total,r15to19,7998.845291,4,41425.75
143,1.0,2016,incomerange,total,r20to24,6627.827408,5,41425.75
144,1.0,2016,incomerange,total,r25to29,5455.642004,6,41425.75
145,1.0,2016,incomerange,total,r30to34,5543.887833,7,41425.75
146,1.0,2016,incomerange,total,r35to39,4955.625670,8,41425.75
147,1.0,2016,incomerange,total,r40to44,3951.242326,9,41425.75
148,1.0,2016,incomerange,total,r45to49,3214.696785,10,41425.75


## Income Range by Household Type

## Old: Aggregate number of households and median hh income

In [ ]:
# Import Census tabular data
census = pd.read_parquet('s3://hcid-cdbg-project-ita-data/data/raw/raw_census_cleaned.parquet')
# Test this on 2016, 2017 for all census tracts
census = census[(census.year>=2016) & (census.table=='income')]

In [ ]:
# Import crosswalks
council_districts = catalog.crosswalk_tracts_council_districts.read()
neighborhood_councils = catalog.crosswalk_tracts_neighborhood_councils.read()
zipcodes = catalog.crosswalk_tracts_zipcodes.read()
congressional_districts = catalog.crosswalk_tracts_congressional_districts.read()
neighborhoods = catalog.crosswalk_tracts_neighborhoods.read()

In [ ]:
# Merge the hh and income portions of the table to calculate the weighted median hh income by tract.
hh = census[census.main_var == 'hh']
income = census[census.main_var == 'medincome']

m1 = pd.merge(hh, income, on = ['GEOID', 'year', 'table', 'second_var'], how = 'left', validate = 'm:1')
m1.rename(columns = {'num_x': 'hh', 'num_y': 'medincome'}, inplace = True)
m1['num'] = m1.hh * m1.medincome
m1 = m1[['GEOID', 'year', 'table', 'second_var', 'hh', 'medincome', 'num']]

In [ ]:
boundaries = {'council_districts': council_districts, 'neighborhood_councils': neighborhood_councils,
             'zipcodes': zipcodes, 'congressional_districts': congressional_districts, 'neighborhoods': neighborhoods}

processed_dfs = {}

for key, value in boundaries.items():
    # Merge the income table with each boundary 
    merged = pd.merge(m1, value, on = 'GEOID', how = 'left', validate = 'm:1')
    merged.max_val = merged.max_val.fillna(0)
    # Allocate the num column according to however many CDs, NCs, etc each tract intersects with. 
    # Find the sum for num1, num2, ... columns. Then, append and take the sum again.
    n = merged.max_val.max().astype(int)
    uniform_id_col = 'ID'
    uniform_num_col = 'num' 
    uniform_pop_col = 'hh'
    aggregated = pd.DataFrame()
    # Depending on the boundary, tract might intersect with 1, 2,...,5 of the larger geographies.
    for i in range(1, n + 1):
        num_col = f"num{i}"
        pop_col = f"hh{i}"
        allocate_col = f"allocate{i}"
        id_col = f"ID{i}"
        # Allocate the num column for all the various intersections.
        merged[num_col] = merged.num * merged[allocate_col]
        merged[pop_col] = merged.hh * merged[allocate_col]
        # Take the sum of the num column by CD, NC, etc.
        agg = merged.groupby([id_col, 'year', 'table', 'second_var']).agg({pop_col: 'sum', num_col: 'sum'}).reset_index()
        agg.rename(columns = {id_col: uniform_id_col, num_col: uniform_num_col, pop_col: uniform_pop_col}, inplace = True)
        # Append these sums together
        aggregated = aggregated.append(agg)
    # Take the sum again. For each CD, NC, etc, calculate the total # of hh and the total hh-weighted income
    aggregated2 = aggregated.groupby([uniform_id_col, 'year', 'table', 'second_var']).agg({uniform_pop_col: 'sum', uniform_num_col: 'sum'}).reset_index()
    processed_dfs[key] = aggregated2

In [ ]:
final_dfs = {}

for key, value in processed_dfs.items():
    df = value.copy()
    # Calculate median hh income 
    df['medincome'] = df.num / df.hh
    # Clean up final_df before saving into dictionary
    for col in ['hh', 'num']:
        df[col] = df[col].round(0).astype(int)
    df['medincome'] = df.medincome.round(2)
    final_dfs[key] = df

In [ ]:
for key, value in final_dfs.items():
    display(key)
    display(value.head())

In [ ]:
test = final_dfs['council_districts']
test = test[(test.year==2017) & (test.ID==6)]

In [ ]:
test